In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split

def convert_1d_to_3d(x):
    return x.reshape((x.shape[0], 25, 25, 1))

def convert_3d_to_1d(x):
    return x.reshape((x.shape[0], 625))

def get_train_dataset(delta, train):
    indexes = train['delta'] ==delta
    end_states = train[indexes].iloc[:, 627:].values
    start_states = train[indexes].iloc[:, 2:627].values
    train_x, eval_x, train_y, eval_y = train_test_split(
        end_states,
        start_states,
        test_size=0.2,
        random_state=42)
    return convert_1d_to_3d(train_x), convert_1d_to_3d(train_y), convert_1d_to_3d(eval_x), convert_1d_to_3d(eval_y)

def get_test_dataset(delta, test):
    indexes = test['delta']==delta
    end_states = test[indexes].iloc[:, 2:].values
    return convert_1d_to_3d(end_states)

In [ ]:
# Data Preparation
train = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/train.csv')
train_xs = []
train_ys = []
eval_xs = []
eval_ys = []
for delta in range(5):
    train_x, train_y, eval_x, eval_y = get_train_dataset(delta + 1, train)
    train_xs.append(train_x)
    train_ys.append(train_y)
    eval_xs.append(eval_x)
    eval_ys.append(eval_y)

In [ ]:
# Model
my_models = []
for delta in range(5):
    name = 'delta_' + str(delta + 1)
    model = tf.keras.models.Sequential(name=name)
    model.add(tf.keras.layers.Conv2D(16, (7, 7), padding='same', kernel_initializer='lecun_normal', activation='selu', strides=(1, 1), input_shape=(25, 25, 1)))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', kernel_initializer='lecun_normal', activation='selu', strides=(1, 1), input_shape=(25, 25, 1)))
    model.add(tf.keras.layers.Dense(25))
    model.add(tf.keras.layers.Dense(1))
    model.add(tf.keras.layers.Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Train
    # https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit
    model.fit(train_xs[delta], train_ys[delta], epochs=10, validation_data=(eval_xs[delta], eval_ys[delta]))
    my_models.append(model)

In [ ]:
# Predict
preds = []
test = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/test.csv')
for delta in range(5):
    test_dataset = get_test_dataset(delta + 1, test)
    pred = my_models[delta].predict(test_dataset)
    preds.append((pred > 0.5).astype(int))

In [ ]:
# Submit
submission = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/sample_submission.csv', index_col='id')
for delta in range(5):
    pred = convert_3d_to_1d(preds[delta])
    indexes = test['delta']== (delta + 1) 
    j = 0
    for i in range(len(indexes)):
        if indexes[i]:
            submission[i:i+1] = pred[j].reshape(1, -1)
            j += 1
submission.to_csv('submission.csv')